# Copy data elements from one instance to another
Do some basic things with metadata using the DHIS2 API

In [1]:
# Load up environment
from dotenv import load_dotenv
import os

load_dotenv()

print('HANDY_DHIS2_USER_SOURCE: {}'.format(os.getenv('HANDY_DHIS2_USER_SOURCE')))
print('HANDY_DHIS2_PASSWORD_SOURCE: {}'.format(os.getenv('HANDY_DHIS2_PASSWORD_SOURCE')))
print('HANDY_DHIS2_HOST_SOURCE: {}'.format(os.getenv('HANDY_DHIS2_HOST_SOURCE')))

print('HANDY_DHIS2_USER_DEST: {}'.format(os.getenv('HANDY_DHIS2_USER_DEST')))
print('HANDY_DHIS2_PASSWORD_DEST: {}'.format(os.getenv('HANDY_DHIS2_PASSWORD_DEST')))
print('HANDY_DHIS2_HOST_DEST: {}'.format(os.getenv('HANDY_DHIS2_HOST_DEST')))

HANDY_DHIS2_USER_SOURCE: admin
HANDY_DHIS2_PASSWORD_SOURCE: Thebig36d5$
HANDY_DHIS2_HOST_SOURCE: https://owod.dhis2.rti-ghd.org
HANDY_DHIS2_USER_DEST: admin
HANDY_DHIS2_PASSWORD_DEST: district
HANDY_DHIS2_HOST_DEST: http://localhost:8086


In [7]:
import requests
import json

try:
    r = requests.get(os.getenv('HANDY_DHIS2_HOST_SOURCE')+'/api/dataElements.json?paging=false&fields=*',
              auth=(os.getenv('HANDY_DHIS2_USER_SOURCE'),os.getenv('HANDY_DHIS2_PASSWORD_SOURCE'))  )
except requests.exceptions.RequestException as e:  # This is the correct syntax
    print(e)

response = r.json()

with open('dataelments.json', 'w') as outfile:
    json.dump(response, outfile)


In [17]:
import json

# Now load this in
f = open('dataelments.json')
data = json.load(f)

dataelements = []
for n in data['dataElements']:
    dataelement = {}

    if n['name'].startswith('Covid'):
        dataelement['id'] = n['id']
        dataelement['name'] = n['name']
        dataelement['shortName'] = n['shortName']
        dataelement['aggregationType'] = n['aggregationType']
        dataelement['domainType'] = n['domainType']
        dataelement['valueType'] = n['valueType']
        dataelements.append(dataelement)
    
# put it in an object
data = {}
data['dataElements'] = dataelements

In [20]:
import requests

# and load this up
try:
    r = requests.post(os.getenv('HANDY_DHIS2_HOST_DEST')+'/api/metadata.json', json=data,
              auth=(os.getenv('HANDY_DHIS2_USER_DEST'),os.getenv('HANDY_DHIS2_PASSWORD_DEST'))  )
except requests.exceptions.RequestException as e:  # This is the correct syntax
    print(e)

r.text

In [19]:
# write out (opttional)
with open('file_for_importing.json', 'w') as outfile:
    json.dump(data, outfile, ensure_ascii=False, indent=4)


In [ ]:
# Here is the curl command if you want
print('curl --user '+os.getenv('HANDY_DHIS2_USER_DEST')+':'+os.getenv('HANDY_DHIS2_PASSWORD_DEST')+' '+os.getenv('HANDY_DHIS2_HOST_DEST') + '/api/metadata.json --header "Content-Type: application/json" --data @file_for_importing.json')